In [ ]:
import pandas as pd

In [ ]:
# download unter https://datanizing.com/llm-tag/arxiv-2023-llm.parquet
ai = pd.read_parquet("arxiv-2023-llm.parquet")

# Sentence splitting

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm.auto import tqdm
ai["sentences"] = None

ai["nouns"] = None
ai["adjectives"] = None
ai["verbs"] = None
ai["lemmas"] = None

ai["entities"] = None
for i, row in tqdm(ai.iterrows(), total=len(ai)):

    nouns = []
    adjectives = []
    verbs = []
    lemmas = []
    nav = []

    doc = nlp(row["title"] + ".\n" + row["abstract"])
    for token in doc:
        lemmas.append(token.lemma_)
        # noun
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            nouns.append(token.lemma_)
            nav.append(token.lemma_)
        # adjective
        if token.pos_ == "ADJ" or token.pos_ == "ADV":
            adjectives.append(token.lemma_)
            nav.append(token.lemma_)
        # verb
        if token.pos_ == "VERB":
            verbs.append(token.lemma_)
            nav.append(token.lemma_)

    sentences = []
    for sentence in doc.sents:
        sentences.append(str(sentence).strip())
        
    ai.at[i, "sentences"] = sentences

    ai.at[i, "nouns"] = nouns
    ai.at[i, "adjectives"] = adjectives
    ai.at[i, "verbs"] = verbs
    ai.at[i, "lemmas"] = lemmas

    ai.at[i, "entities"] = list([str(e) for e in doc.ents])

In [ ]:
ai.to_parquet("arxiv-2023-llm-linguistic.parquet", compression="zstd")